# MLSP 2013 Birds – Production Notebook

Goal: Build a strong, clean, reproducible pipeline to reach ≥0.87 AUC (bronze+) using fast tabular features and robust CV.

Plan:
- Data: parse folds, rec->filename, labels (from rec_labels_test_hidden.txt, with all-negative allowed).
- Features:
  - Aggregate supplemental_data/segment_features.txt + segment_rectangles.txt per rec_id.
  - Fast aggregations only: mean, std, min, max, median, count.
  - Engineer: duration, freq_span, area, coverage_ratio (time), segment_density, freq coverage.
  - Per-fold station priors.
  - Per-fold SVD (n_components=16-24) on L1-normalized histogram_of_segments.txt.
- Validation: GroupKFold(n_splits=3) by station.
- Model: LightGBM (learning_rate 0.02, num_leaves 31, n_estimators 3000, early_stopping 120).
- Seed averaging: 3 seeds; save strict ID orders and OOF/test preds.
- Next: add XGBoost and LR models on same features + simple meta-learner blend.

In [45]:
# Setup, data parsing, and helpers (strict ID alignment ready)
import os, sys, time, glob, re, math, json, gc
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import TruncatedSVD

t0 = time.time()
BASE = Path.cwd()
print('CWD:', BASE)

# 1) Load core metadata
sp_path = BASE/'essential_data'/'species_list.txt'
cv_path = BASE/'essential_data'/'CVfolds_2.txt'
map_path = BASE/'essential_data'/'rec_id2filename.txt'
lab_path = BASE/'essential_data'/'rec_labels_test_hidden.txt'
hos_path = BASE/'supplemental_data'/'histogram_of_segments.txt'
segf_path = BASE/'supplemental_data'/'segment_features.txt'
segr_path = BASE/'supplemental_data'/'segment_rectangles.txt'

sp_df = pd.read_csv(sp_path)
num_classes = int(sp_df.shape[0])
cv_df = pd.read_csv(cv_path)
rec_map = pd.read_csv(map_path)
# Robust station parsing via regex
rec_map['station'] = rec_map['filename'].astype(str).str.extract(r'^([A-Z0-9]+)_', expand=False).fillna('UNK')

# Extract temporal metadata from filename: YYYYMMDD_HHMMSS
def parse_time_parts(fn: str):
    m = re.search(r'_(\d{8})_(\d{6})_', fn)
    if not m:
        return np.int16(-1), np.int16(-1), np.int16(-1)
    ymd, hms = m.group(1), m.group(2)
    try:
        month = int(ymd[4:6])
        hour = int(hms[0:2])
        # hour bins: dawn(4-7)=0, day(8-16)=1, dusk(17-20)=2, night(21-3)=3
        if 4 <= hour <= 7: hb = 0
        elif 8 <= hour <= 16: hb = 1
        elif 17 <= hour <= 20: hb = 2
        else: hb = 3
        return np.int16(month), np.int16(hour), np.int16(hb)
    except:
        return np.int16(-1), np.int16(-1), np.int16(-1)

rec_map[['month','hour','hour_bin']] = rec_map['filename'].astype(str).apply(lambda s: pd.Series(parse_time_parts(s)))

meta = cv_df.merge(rec_map, on='rec_id', how='left')
print('Species:', num_classes, '| meta shape:', meta.shape)

# 2) Parse labels with all-negative allowed; process lines and skip only '?' tokens
label_rows = []
with open(lab_path, 'r') as f:
    _ = next(f, None)
    for line in f:
        s = line.strip()
        if not s: continue
        parts = [p for p in s.split(',')]
        try:
            rid = int(parts[0])
        except:
            continue
        y = np.zeros(num_classes, dtype=np.int8)
        # tokens after rec_id can be class ids, '?' or blanks
        for p in parts[1:]:
            p = p.strip()
            if p == '' or p == '?':
                continue
            try:
                cid = int(p)
            except:
                continue
            if 0 <= cid < num_classes:
                y[cid] = 1
        label_rows.append((rid, y))
label_map = {rid:y for rid, y in label_rows}
print('Labeled rec_ids:', len(label_map))

# 3) Robust parse for histogram_of_segments with RAW counts preserved
hist_rows = []
with open(hos_path, 'r') as f:
    _ = next(f, None)
    for line in f:
        s = line.strip()
        if not s: continue
        parts = [p for p in s.split(',') if p!='']
        try:
            rid = int(parts[0])
        except:
            continue
        vals = []
        for v in parts[1:]:
            try: vals.append(float(v))   # RAW counts
            except: vals.append(0.0)
        hist_rows.append((rid, vals))
max_len_h = max((len(v) for _, v in hist_rows), default=0)
Hc = np.zeros((len(hist_rows), max_len_h), dtype=np.float32)  # RAW counts
R = np.zeros((len(hist_rows),), dtype=int)
for i, (rid, vals) in enumerate(hist_rows):
    R[i] = rid
    L = min(max_len_h, len(vals))
    if L: Hc[i, :L] = np.asarray(vals[:L], np.float32)

hist_df_counts = pd.DataFrame(Hc)
hist_df_counts.insert(0, 'rec_id', R)
# De-duplicate by rec_id using mean aggregation
dup_count = int(hist_df_counts.duplicated('rec_id').sum())
if dup_count > 0:
    print('Histogram duplicate rec_ids:', dup_count, '-> aggregating by mean')
hist_df_counts = hist_df_counts.groupby('rec_id', as_index=False).mean()
print('Histogram (counts) matrix (deduped):', hist_df_counts.shape)

# Build L1-normalized histogram from counts
H_counts = hist_df_counts.iloc[:, 1:].to_numpy(np.float32)
row_sums = H_counts.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1.0
H_l1_all = H_counts / row_sums

# 4) Parse segment features and rectangles (fast)
seg_records = []
with open(segf_path, 'r') as f:
    _ = next(f, None)
    for line in f:
        s = line.strip()
        if not s: continue
        parts = [p for p in s.split(',') if p!='']
        if len(parts) < 3: continue
        try:
            rid = int(parts[0]); seg = int(parts[1])
        except:
            continue
        vals = []
        for v in parts[2:]:
            try: vals.append(float(v))
            except: vals.append(0.0)
        seg_records.append((rid, seg, vals))
max_len_sf = max((len(v) for _,_,v in seg_records), default=0)
sf_cols = [f'sf_{i}' for i in range(max_len_sf)]
sf_df = (pd.DataFrame([([rid, seg] + v + [0.0]*(max_len_sf - len(v)))
                        for rid, seg, v in seg_records],
                       columns=['rec_id','seg_idx']+sf_cols) if seg_records else
         pd.DataFrame(columns=['rec_id','seg_idx']+sf_cols))
print('segment_features:', sf_df.shape)

rect_rows = []
with open(segr_path, 'r') as f:
    _ = next(f, None)
    for line in f:
        s = line.strip().strip(',')
        if not s: continue
        parts = [p for p in s.split(',') if p!='']
        if len(parts) < 6: continue
        try:
            rid = int(parts[0]); seg = int(parts[1])
            t0r = float(parts[2]); t1r = float(parts[3]); f0r = float(parts[4]); f1r = float(parts[5])
        except:
            continue
        rect_rows.append((rid, seg, t0r, t1r, f0r, f1r))
rect_df = pd.DataFrame(rect_rows, columns=['rec_id','seg_idx','t_start','t_end','f_start','f_end'])
if not rect_df.empty:
    rect_df['duration'] = rect_df['t_end'] - rect_df['t_start']
    rect_df['freq_span'] = rect_df['f_end'] - rect_df['f_start']
    rect_df['area_tf'] = rect_df['duration'] * rect_df['freq_span']
print('segment_rectangles:', rect_df.shape)

# 5) Merge per-segment and fast aggregate
seg_full = (sf_df.merge(rect_df, on=['rec_id','seg_idx'], how='left') if not rect_df.empty else sf_df.copy())
seg_full['segment_count'] = 1
num_cols = [c for c in seg_full.columns if c not in ['rec_id','seg_idx']]
agg_funcs = ['mean','std','min','max','median']
agg_dict = {c: agg_funcs for c in num_cols}
gb = seg_full.groupby('rec_id').agg(agg_dict) if not seg_full.empty else pd.DataFrame()
if not gb.empty:
    gb.columns = [f"{a}_{b}" for a,b in gb.columns.to_flat_index()]
    gb = gb.reset_index()
    # segment_count_total keeps total count; fill only this with 0 later
    gb['segment_count_total'] = seg_full.groupby('rec_id')['segment_count'].sum().values
else:
    gb = pd.DataFrame({'rec_id': meta['rec_id'].unique()})

# 6) Build base feature frame (no global fillna!), add engineered features
feat_df = meta.merge(gb, on='rec_id', how='left')

# has_segments flag and selective filling
feat_df['has_segments'] = (~feat_df['segment_count_total'].isna()).astype(np.int8)
feat_df['segment_count_total'] = feat_df['segment_count_total'].fillna(0.0)

# Engineered record-level features (10s clips); NaN-safe operations
def safe_mul(a, b):
    return a*b
dur_mean_col = 'duration_mean'
freq_span_mean_col = 'freq_span_mean'
area_mean_col = 'area_tf_mean'
f_end_max_col = 'f_end_max'
f_start_min_col = 'f_start_min'

if dur_mean_col in feat_df.columns:
    feat_df['coverage_ratio'] = (feat_df['segment_count_total'] * feat_df[dur_mean_col]) / 10.0
    feat_df['log1p_duration_mean'] = np.log1p(feat_df[dur_mean_col])
else:
    feat_df['coverage_ratio'] = np.nan
    feat_df['log1p_duration_mean'] = np.nan

feat_df['segment_density'] = feat_df['segment_count_total'] / 10.0

if area_mean_col in feat_df.columns:
    feat_df['log1p_area_tf_mean'] = np.log1p(feat_df[area_mean_col])
else:
    feat_df['log1p_area_tf_mean'] = np.nan

feat_df['log1p_segment_count_total'] = np.log1p(feat_df['segment_count_total'])

if (freq_span_mean_col in feat_df.columns) and (dur_mean_col in feat_df.columns):
    feat_df['dur_x_freqspan'] = safe_mul(feat_df[dur_mean_col], feat_df[freq_span_mean_col])
else:
    feat_df['dur_x_freqspan'] = np.nan

if (f_end_max_col in feat_df.columns) and (f_start_min_col in feat_df.columns):
    feat_df['freq_coverage'] = (feat_df[f_end_max_col] - feat_df[f_start_min_col]) / 8000.0
else:
    feat_df['freq_coverage'] = np.nan

# Curated rectangle/meta features list
curated_cols = [
    'segment_count_total', 'coverage_ratio', 'duration_mean', 'area_tf_mean', 'freq_span_mean', 'freq_coverage',
    'log1p_duration_mean', 'log1p_area_tf_mean', 'log1p_segment_count_total',
    'month', 'hour', 'hour_bin'
]
for c in curated_cols:
    if c not in feat_df.columns:
        feat_df[c] = np.nan

# Split masks
known_mask = feat_df['rec_id'].isin(label_map.keys())
train_mask = (feat_df['fold']==0) & known_mask
test_mask = (feat_df['fold']==1)

# Save ID orders for strict alignment downstream
rec_train = feat_df.loc[train_mask, 'rec_id'].values.astype(int)
rec_test = feat_df.loc[test_mask, 'rec_id'].values.astype(int)
np.save('prod_train_ids.npy', rec_train)
np.save('prod_test_ids.npy', rec_test)

# Feature columns (exclude meta)
exclude_cols = set(['rec_id','fold','filename','station'])
feat_cols = [c for c in feat_df.columns if c not in exclude_cols]

# Prepare arrays
X_base = feat_df.loc[train_mask, feat_cols].to_numpy(np.float32)
Y_train = np.vstack([label_map[int(r)] for r in rec_train]).astype(np.int8)
X_test_base = feat_df.loc[test_mask, feat_cols].to_numpy(np.float32)
groups = feat_df.loc[train_mask, 'station'].astype(str).values
stations_test = feat_df.loc[test_mask, 'station'].astype(str).values

# Map rec_id -> histogram row (post-dedup COUNT matrix) with assertions
rid_to_hrow = {int(r): i for i, r in enumerate(hist_df_counts['rec_id'].values)}
missing_train = [int(r) for r in rec_train if int(r) not in rid_to_hrow]
missing_test = [int(r) for r in rec_test if int(r) not in rid_to_hrow]
assert len(missing_train) == 0, f'Missing histogram rows for train rec_ids: {missing_train[:10]} (+{len(missing_train)-10 if len(missing_train)>10 else 0})'
assert len(missing_test) == 0, f'Missing histogram rows for test rec_ids: {missing_test[:10]} (+{len(missing_test)-10 if len(missing_test)>10 else 0})'
Hc_train = np.vstack([hist_df_counts.iloc[rid_to_hrow[int(r)]].values[1:] for r in rec_train]).astype(np.float32)
Hc_test = np.vstack([hist_df_counts.iloc[rid_to_hrow[int(r)]].values[1:] for r in rec_test]).astype(np.float32)

# Helper: L1 normalize
def l1_norm(X):
    s = X.sum(axis=1, keepdims=True)
    s[s==0] = 1.0
    return X / s

# Helper: Gaussian smoothing with sigma~1.5 across bins
def gaussian_kernel1d(sigma=1.5, radius=None):
    if radius is None:
        radius = int(max(1, round(3*sigma)))
    x = np.arange(-radius, radius+1, dtype=np.float32)
    k = np.exp(-(x**2)/(2*(sigma**2)))
    k /= k.sum() if k.sum()!=0 else 1.0
    return k

def smooth_rows(X, sigma=1.5):
    k = gaussian_kernel1d(sigma=sigma)
    rad = (len(k)-1)//2
    # pad reflect at edges and convolve per row
    Xp = np.pad(X, ((0,0),(rad,rad)), mode='reflect')
    out = np.empty_like(X)
    for i in range(X.shape[0]):
        out[i] = np.convolve(Xp[i], k, mode='valid')
    return out

# 7) Histogram variants
# H1: L1-normalized counts
H1_train = l1_norm(Hc_train)
H1_test = l1_norm(Hc_test)
# Smoothing
H1s_train = smooth_rows(H1_train, sigma=1.5)
H1s_test = smooth_rows(H1_test, sigma=1.5)
# Drop first and last bins
H1sd_train = H1s_train[:, 1:-1].astype(np.float32)
H1sd_test = H1s_test[:, 1:-1].astype(np.float32)
# IDF weighting
df_bins = (Hc_train > 0).sum(axis=0).astype(np.float32)  # on train only
N_docs = float(Hc_train.shape[0]) if Hc_train.shape[0]>0 else 1.0
idf = np.log((N_docs + 1.0) / (df_bins + 1.0)).astype(np.float32)
idf = idf[1:-1]  # align with dropped edges
H_idf_train = H1sd_train * idf[None, :]
H_idf_test = H1sd_test * idf[None, :]  # use train-based idf for test
H_idf_train = l1_norm(H_idf_train).astype(np.float32)
H_idf_test = l1_norm(H_idf_test).astype(np.float32)

# H_tree: sqrt(raw_counts) then row-normalize (for trees); drop edges too
Hsq_train = np.sqrt(np.clip(Hc_train, 0.0, None))
Hsq_test = np.sqrt(np.clip(Hc_test, 0.0, None))
H_tree_train = l1_norm(Hsq_train)[:, 1:-1].astype(np.float32)
H_tree_test = l1_norm(Hsq_test)[:, 1:-1].astype(np.float32)

# Keep previous raw/hellinger arrays for backward compatibility
H_train = H1_train.astype(np.float32)    # previous RAW L1 version
H_test = H1_test.astype(np.float32)
H_train_h = np.sqrt(np.clip(H_train, 0.0, None)).astype(np.float32)
H_test_h = np.sqrt(np.clip(H_test, 0.0, None)).astype(np.float32)

# Curated features arrays (with simple imputation & clipping)
cur_df_tr = feat_df.loc[train_mask, curated_cols].copy()
cur_df_te = feat_df.loc[test_mask, curated_cols].copy()
for c in curated_cols:
    if cur_df_tr[c].dtype.kind in 'biu':
        cur_df_tr[c] = cur_df_tr[c].fillna(-1)
        cur_df_te[c] = cur_df_te[c].fillna(-1)
    else:
        med = cur_df_tr[c].median() if np.isfinite(cur_df_tr[c].median()) else 0.0
        cur_df_tr[c] = cur_df_tr[c].fillna(med)
        cur_df_te[c] = cur_df_te[c].fillna(med)
        q99 = np.nanpercentile(cur_df_tr[c].values, 99) if cur_df_tr.shape[0]>0 else None
        if q99 is not None and np.isfinite(q99):
            cur_df_tr[c] = np.clip(cur_df_tr[c].values, None, q99)
            cur_df_te[c] = np.clip(cur_df_te[c].values, None, q99)
C_train = cur_df_tr.to_numpy(np.float32)
C_test = cur_df_te.to_numpy(np.float32)

print('X_base:', X_base.shape, '| Y:', Y_train.shape, '| X_test_base:', X_test_base.shape)
print('H1 L1:', H_train.shape, '| H_idf:', H_idf_train.shape, '| H_tree:', H_tree_train.shape, '| Curated:', C_train.shape)
print('Elapsed setup: %.2fs' % (time.time()-t0))

# Helper: compute macro AUC ignoring degenerate classes
def macro_auc_ignoring_degenerate(y_true, y_pred):
    aucs = []
    for c in range(y_true.shape[1]):
        y = y_true[:, c]
        p = y_pred[:, c]
        if y.sum()==0 or y.sum()==y.shape[0]:
            continue
        try:
            aucs.append(roc_auc_score(y, p))
        except:
            pass
    return float(np.mean(aucs)) if aucs else float('nan')

CWD: /app/agent_run_states/mlsp-2013-birds-spray-20250908-000706
Species: 19 | meta shape: (322, 7)
Labeled rec_ids: 322
Histogram (counts) matrix (deduped): (322, 101)
segment_features: (1119, 40)
segment_rectangles: (1119, 9)


X_base: (258, 242) | Y: (258, 19) | X_test_base: (64, 242)
H1 L1: (258, 100) | H_idf: (258, 98) | H_tree: (258, 98) | Curated: (258, 12)
Elapsed setup: 0.27s


In [15]:
# Strong single model: LGBM with GroupKFold(n_splits=3), per-fold station priors + SVD on hist, 3-seed averaging
import sys, subprocess, time
try:
    import lightgbm as lgb
    from lightgbm import LGBMClassifier
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'lightgbm'])
    import lightgbm as lgb
    from lightgbm import LGBMClassifier

from sklearn.model_selection import GroupKFold
from sklearn.decomposition import TruncatedSVD

print('Starting LGBM training... lgb version:', getattr(lgb, '__version__', 'unknown'))

# Base matrices
X_base_df = pd.DataFrame(X_base, columns=feat_cols)
X_test_df = pd.DataFrame(X_test_base, columns=feat_cols)
X_base_mat = X_base_df.to_numpy(np.float32)
X_test_mat = X_test_df.to_numpy(np.float32)

gkf = GroupKFold(n_splits=3)
n_train = X_base_df.shape[0]
n_test = X_test_df.shape[0]

def run_seed(seed):
    oof = np.zeros((n_train, num_classes), dtype=np.float32)
    te = np.zeros((n_test, num_classes), dtype=np.float32)
    for c in range(num_classes):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_train, dtype=np.float32)
        cls_te_acc = np.zeros(n_test, dtype=np.float32)
        fold_no = 0
        for tr_idx, va_idx in gkf.split(X_base_df, y, groups):
            fold_no += 1
            tstart = time.time()
            Xb_tr = X_base_mat[tr_idx]; Xb_va = X_base_mat[va_idx]
            y_tr = y[tr_idx]; y_va = y[va_idx]
            # Station prior within fold with Bayesian smoothing
            st_tr = groups[tr_idx]; st_va = groups[va_idx]
            alpha = 5.0
            global_mean = float(y_tr.mean()) if y_tr.size>0 else 0.0
            st_prior = {}
            for s in np.unique(st_tr):
                mask = (st_tr == s)
                cnt = int(mask.sum())
                sm = float(y_tr[mask].sum()) if cnt>0 else 0.0
                st_prior[s] = (sm + alpha*global_mean) / (cnt + alpha)
            st_tr_feat = np.array([st_prior.get(s, global_mean) for s in st_tr], dtype=np.float32)[:, None]
            st_va_feat = np.array([st_prior.get(s, global_mean) for s in st_va], dtype=np.float32)[:, None]
            st_te_feat = np.array([st_prior.get(s, global_mean) for s in stations_test], dtype=np.float32)[:, None]
            # Histogram features
            H_tr = H_train[tr_idx]; H_va = H_train[va_idx]
            # Per-fold SVD on histogram features (reduced components)
            n_comp = 0
            if H_tr.size > 0:
                n_comp = int(min(16, max(2, min(H_tr.shape[1], max(2, H_tr.shape[0]-1)))))
            if n_comp >= 2:
                svd = TruncatedSVD(n_components=n_comp, random_state=seed)
                svd.fit(H_tr)
                Z_tr = svd.transform(H_tr).astype(np.float32)
                Z_va = svd.transform(H_va).astype(np.float32)
                Z_te = svd.transform(H_test).astype(np.float32)
            else:
                Z_tr = None; Z_va = None; Z_te = None
            # Build fold-wise matrices (base + st_prior + SVD)  [drop raw histogram to reduce sparsity]
            parts_tr = [Xb_tr, st_tr_feat]
            parts_va = [Xb_va, st_va_feat]
            parts_te = [X_test_mat, st_te_feat]
            if Z_tr is not None:
                parts_tr.append(Z_tr); parts_va.append(Z_va); parts_te.append(Z_te)
            X_tr_mat = np.concatenate(parts_tr, axis=1).astype(np.float32)
            X_va_mat = np.concatenate(parts_va, axis=1).astype(np.float32)
            X_te_mat = np.concatenate(parts_te, axis=1).astype(np.float32)
            # Handle degeneracy
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_test, const, np.float32) / gkf.get_n_splits() / 1.0
                print(f'Class {c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-tstart:.2f}s'); sys.stdout.flush()
                continue
            spw = neg / max(1, pos)
            model = LGBMClassifier(
                objective='binary',
                n_estimators=8000,
                learning_rate=0.03,
                scale_pos_weight=spw,
                random_state=seed,
                n_jobs=-1,
                # --- Critical Fixes ---
                feature_pre_filter=False,
                num_leaves=31,
                max_depth=6,
                max_bin=64,
                min_child_samples=1,
                min_sum_hessian_in_leaf=1e-5,
                # --- Stability & Correctness ---
                force_col_wise=True,
                zero_as_missing=False,
                extra_trees=True,
                # --- Standard Parameters ---
                subsample=0.9,
                subsample_freq=1,
                colsample_bytree=0.9,
                reg_lambda=0.1,
                min_gain_to_split=0.0
            )
            model.fit(
                X_tr_mat, y_tr,
                eval_set=[(X_va_mat, y_va)],
                eval_metric='auc',
                callbacks=[lgb.early_stopping(150, verbose=False), lgb.log_evaluation(50)]
            )
            p_va = model.predict_proba(X_va_mat)[:,1].astype(np.float32)
            p_te = model.predict_proba(X_te_mat)[:,1].astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / gkf.get_n_splits()
            print(f'Class {c:02d} fold {fold_no}: pos={pos} neg={neg} spw={spw:.2f} best_iter={getattr(model, "best_iteration_", None)} | {time.time()-tstart:.2f}s'); sys.stdout.flush()
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

seeds = [42, 7, 2025]
oofs = []; tests = []
for i, sd in enumerate(seeds, 1):
    print(f'== Seed {sd} ({i}/{len(seeds)}) =='); sys.stdout.flush()
    oof_s, te_s = run_seed(sd)
    oofs.append(oof_s); tests.append(te_s)

oof_mean = np.mean(np.stack(oofs, axis=0), axis=0)
te_mean = np.mean(np.stack(tests, axis=0), axis=0)

# Evaluate macro AUC
auc = macro_auc_ignoring_degenerate(Y_train, oof_mean)
print(f'Final LGBM (3-seed) OOF Macro AUC: {auc:.5f}')

# Save predictions and ids
np.save('prod_lgb_oof.npy', oof_mean)
np.save('prod_lgb_test.npy', te_mean)
np.save('prod_lgb_train_ids.npy', rec_train)
np.save('prod_lgb_test_ids.npy', rec_test)
print('Saved prod_lgb_* files.')

In [4]:
# Evaluate saved prod_lgb OOF and build submission
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

oof = np.load('prod_lgb_oof.npy')
te = np.load('prod_lgb_test.npy')
tr_ids = np.load('prod_lgb_train_ids.npy').astype(int)
te_ids = np.load('prod_lgb_test_ids.npy').astype(int)

# Build Y in the same order as tr_ids using label_map from setup cell
Y_eval = np.vstack([label_map[int(r)] for r in tr_ids]).astype(np.int8)
auc = macro_auc_ignoring_degenerate(Y_eval, oof)
print(f'prod_lgb OOF Macro AUC (ID-aligned): {auc:.5f}')

# Create submission aligned to sample_submission Id encoding (rec_id*100+class_id)
sub = pd.read_csv('sample_submission.csv')
id_vals = sub['Id'].values.astype(int)
rid_to_idx = {rid:i for i, rid in enumerate(te_ids)}
probs = np.zeros_like(id_vals, dtype=np.float32)
for i, Id in enumerate(id_vals):
    rid = Id // 100; cid = Id % 100
    row = rid_to_idx.get(rid, None)
    p = float(te[row, cid]) if (row is not None and cid < num_classes) else 0.0
    probs[i] = np.clip(p, 1e-6, 1-1e-6)
sub['Probability'] = probs
sub.to_csv('submission_prod_lgb.csv', index=False)
print('Saved submission_prod_lgb.csv')

prod_lgb OOF Macro AUC (ID-aligned): 0.60784
Saved submission_prod_lgb.csv


In [32]:
# Strong single model (alternative): XGBoost with GroupKFold, station priors + SVD + RAW HIST, 3-seed averaging
import sys, subprocess, time, os
from pathlib import Path
try:
    import xgboost as xgb
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import TruncatedSVD

print('Starting XGBoost training... xgb version:', getattr(xgb, '__version__', 'unknown'))

# Clean stale artifacts BEFORE training to avoid evaluating partial runs
for fn in ['prod_xgb_oof.npy','prod_xgb_test.npy','prod_xgb_train_ids.npy','prod_xgb_test_ids.npy']:
    fp = Path(fn)
    if fp.exists():
        try: fp.unlink()
        except Exception as e: print('Could not delete', fn, e)

X_base_df = pd.DataFrame(X_base, columns=feat_cols)
X_test_df = pd.DataFrame(X_test_base, columns=feat_cols)
X_base_mat = X_base_df.to_numpy(np.float32)
X_test_mat = X_test_df.to_numpy(np.float32)

gkf = GroupKFold(n_splits=3)
n_train = X_base_df.shape[0]
n_test = X_test_df.shape[0]

def run_seed_xgb(seed):
    oof = np.zeros((n_train, num_classes), dtype=np.float32)
    te = np.zeros((n_test, num_classes), dtype=np.float32)
    for c in range(num_classes):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_train, dtype=np.float32)
        cls_te_acc = np.zeros(n_test, dtype=np.float32)
        fold_no = 0
        for tr_idx, va_idx in gkf.split(X_base_df, y, groups):
            fold_no += 1
            tstart = time.time()
            # Base dropped per expert advice; use only station prior + RAW hist (+ SVD)
            y_tr = y[tr_idx]; y_va = y[va_idx]
            # Station prior within fold with Bayesian smoothing
            st_tr = groups[tr_idx]; st_va = groups[va_idx]
            alpha = 5.0
            global_mean = float(y_tr.mean()) if y_tr.size>0 else 0.0
            st_prior = {}
            for s in np.unique(st_tr):
                m = (st_tr == s)
                cnt = int(m.sum())
                sm = float(y_tr[m].sum()) if cnt>0 else 0.0
                st_prior[s] = (sm + alpha*global_mean) / (cnt + alpha)
            st_tr_feat = np.array([st_prior.get(s, global_mean) for s in st_tr], dtype=np.float32)[:, None]
            st_va_feat = np.array([st_prior.get(s, global_mean) for s in st_va], dtype=np.float32)[:, None]
            st_te_feat = np.array([st_prior.get(s, global_mean) for s in stations_test], dtype=np.float32)[:, None]
            # Histogram features (RAW + per-fold SVD on RAW)
            H_tr = H_train[tr_idx]; H_va = H_train[va_idx]
            n_comp = 0
            if H_tr.size > 0:
                n_comp = int(min(12, max(2, min(H_tr.shape[1], max(2, H_tr.shape[0]-1)))))
            if n_comp >= 2:
                svd = TruncatedSVD(n_components=n_comp, random_state=seed)
                svd.fit(H_tr)
                Z_tr = svd.transform(H_tr).astype(np.float32)
                Z_va = svd.transform(H_va).astype(np.float32)
                Z_te = svd.transform(H_test).astype(np.float32)
            else:
                Z_tr = None; Z_va = None; Z_te = None
            # Build matrices: STATION PRIOR + SVD + RAW HIST (drop X_base)
            parts_tr = [st_tr_feat]
            parts_va = [st_va_feat]
            parts_te = [st_te_feat]
            if Z_tr is not None:
                parts_tr.append(Z_tr); parts_va.append(Z_va); parts_te.append(Z_te)
            parts_tr.append(H_tr); parts_va.append(H_va); parts_te.append(H_test)
            X_tr_mat = np.concatenate(parts_tr, axis=1).astype(np.float32)
            X_va_mat = np.concatenate(parts_va, axis=1).astype(np.float32)
            X_te_mat = np.concatenate(parts_te, axis=1).astype(np.float32)
            print(f'[XGB] Class {c:02d} fold {fold_no} shapes: X_tr={X_tr_mat.shape} X_va={X_va_mat.shape}', flush=True)
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_test, const, np.float32) / gkf.get_n_splits()
                print(f'[XGB] Class {c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-tstart:.2f}s'); sys.stdout.flush()
                continue
            spw = neg / max(1, pos)
            dtrain = xgb.DMatrix(X_tr_mat, label=y_tr)
            dvalid = xgb.DMatrix(X_va_mat, label=y_va)
            dtest = xgb.DMatrix(X_te_mat)
            params = {
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'max_depth': 4,
                'min_child_weight': 1e-3,
                'eta': 0.02,
                'subsample': 0.9,
                'colsample_bytree': 0.8,
                'lambda': 0.1,
                'tree_method': 'hist',
                'max_bin': 64,
                'scale_pos_weight': float(spw),
                'seed': int(seed),
                'verbosity': 1
            }
            evallist = [(dvalid, 'valid')]
            bst = xgb.train(params, dtrain, num_boost_round=6000, evals=evallist,
                            early_stopping_rounds=200, verbose_eval=100)
            p_va = bst.predict(dvalid, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            p_te = bst.predict(dtest, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / gkf.get_n_splits()
            print(f'[XGB] Class {c:02d} fold {fold_no}: pos={pos} neg={neg} spw={spw:.2f} best_iter={bst.best_iteration} | {time.time()-tstart:.2f}s'); sys.stdout.flush()
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

seeds = [42, 7, 2025]
oofs = []; tests = []
for i, sd in enumerate(seeds, 1):
    print(f'== XGB Seed {sd} ({i}/{len(seeds)}) =='); sys.stdout.flush()
    oof_s, te_s = run_seed_xgb(sd)
    oofs.append(oof_s); tests.append(te_s)

oof_mean = np.mean(np.stack(oofs, axis=0), axis=0)
te_mean = np.mean(np.stack(tests, axis=0), axis=0)
auc = macro_auc_ignoring_degenerate(Y_train, oof_mean)
print(f'Final XGBoost (3-seed) OOF Macro AUC: {auc:.5f}')

# Save with temp names then rename for integrity
tmp_oof = 'prod_xgb_oof_tmp.npy'; tmp_te = 'prod_xgb_test_tmp.npy'
np.save(tmp_oof, oof_mean); np.save(tmp_te, te_mean)
np.save('prod_xgb_train_ids.npy', rec_train)
np.save('prod_xgb_test_ids.npy', rec_test)
os.replace(tmp_oof, 'prod_xgb_oof.npy')
os.replace(tmp_te, 'prod_xgb_test.npy')
print('Saved prod_xgb_* files.')

Starting XGBoost training... xgb version: 2.1.4
== XGB Seed 42 (1/3) ==


[XGB] Class 00 fold 1 shapes: X_tr=(167, 113) X_va=(91, 113)


[0]	valid-auc:0.62500


[100]	valid-auc:0.97989


[200]	valid-auc:0.97989


...

[100]	valid-auc:0.92393


[200]	valid-auc:0.92022


[299]	valid-auc:0.91466


[XGB] Class 18 fold 3: pos=5 neg=169 spw=33.80 best_iter=99 | 0.39s


Final XGBoost (3-seed) OOF Macro AUC: 0.61451
Saved prod_xgb_* files.


In [33]:
# Evaluate saved prod_xgb OOF and build submission
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

oof = np.load('prod_xgb_oof.npy')
te = np.load('prod_xgb_test.npy')
tr_ids = np.load('prod_xgb_train_ids.npy').astype(int)
te_ids = np.load('prod_xgb_test_ids.npy').astype(int)

# Build Y in the same order as tr_ids using label_map from setup cell
Y_eval = np.vstack([label_map[int(r)] for r in tr_ids]).astype(np.int8)
auc = macro_auc_ignoring_degenerate(Y_eval, oof)
print(f'prod_xgb OOF Macro AUC (ID-aligned): {auc:.5f}')

# Create submission aligned to sample_submission Id encoding (rec_id*100+class_id)
sub = pd.read_csv('sample_submission.csv')
id_vals = sub['Id'].values.astype(int)
rid_to_idx = {rid:i for i, rid in enumerate(te_ids)}
probs = np.zeros_like(id_vals, dtype=np.float32)
for i, Id in enumerate(id_vals):
    rid = Id // 100; cid = Id % 100
    row = rid_to_idx.get(rid, None)
    p = float(te[row, cid]) if (row is not None and cid < num_classes) else 0.0
    probs[i] = np.clip(p, 1e-6, 1-1e-6)
sub['Probability'] = probs
sub.to_csv('submission_prod_xgb.csv', index=False)
print('Saved submission_prod_xgb.csv')

prod_xgb OOF Macro AUC (ID-aligned): 0.61451
Saved submission_prod_xgb.csv


In [22]:
# Sanity baseline: LogisticRegression on [station prior + RAW histogram] with GroupKFold; fast OOF check
import os
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

print('Starting LogisticRegression sanity baseline...')

gkf = GroupKFold(n_splits=3)
n_train = Y_train.shape[0]; n_test = X_test_base.shape[0]
num_classes_local = Y_train.shape[1]

def run_seed_lr(seed):
    oof = np.zeros((n_train, num_classes_local), dtype=np.float32)
    te = np.zeros((n_test, num_classes_local), dtype=np.float32)
    for c in range(num_classes_local):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_train, dtype=np.float32)
        cls_te_acc = np.zeros(n_test, dtype=np.float32)
        fold_no = 0
        for tr_idx, va_idx in gkf.split(X_base, y, groups):
            fold_no += 1
            # Build fold-wise features: station prior (1-d) + RAW histogram bins (101 dims)
            st_tr = groups[tr_idx]; st_va = groups[va_idx]
            alpha = 5.0
            y_tr = y[tr_idx]; y_va = y[va_idx]
            gm = float(y_tr.mean()) if y_tr.size>0 else 0.0
            st_prior = {}
            for s in np.unique(st_tr):
                m = (st_tr == s)
                cnt = int(m.sum())
                sm = float(y_tr[m].sum()) if cnt>0 else 0.0
                st_prior[s] = (sm + alpha*gm) / (cnt + alpha)
            st_tr_feat = np.array([st_prior.get(s, gm) for s in st_tr], dtype=np.float32)[:, None]
            st_va_feat = np.array([st_prior.get(s, gm) for s in st_va], dtype=np.float32)[:, None]
            st_te_feat = np.array([st_prior.get(s, gm) for s in stations_test], dtype=np.float32)[:, None]
            H_tr = H_train[tr_idx]; H_va = H_train[va_idx]
            # Concatenate
            X_tr = np.concatenate([st_tr_feat, H_tr], axis=1).astype(np.float32)
            X_va = np.concatenate([st_va_feat, H_va], axis=1).astype(np.float32)
            X_te = np.concatenate([st_te_feat, H_test], axis=1).astype(np.float32)
            # Degenerate handling
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_test, const, np.float32) / gkf.get_n_splits()
                print(f'[LR] Class {c:02d} fold {fold_no}: degenerate -> const {const:.4f}')
                continue
            # Scale features
            scaler = StandardScaler(with_mean=True, with_std=True)
            X_tr_s = scaler.fit_transform(X_tr)
            X_va_s = scaler.transform(X_va)
            X_te_s = scaler.transform(X_te)
            # LR with class balance
            lr = LogisticRegression(
                penalty='l2', C=0.1,
                class_weight='balanced',
                solver='liblinear',
                max_iter=1000,
                random_state=seed
            )
            lr.fit(X_tr_s, y_tr)
            p_va = lr.predict_proba(X_va_s)[:,1].astype(np.float32)
            p_te = lr.predict_proba(X_te_s)[:,1].astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / gkf.get_n_splits()
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

seeds = [42]
oofs = []; tests = []
for sd in seeds:
    print(f'== LR Seed {sd} ==')
    oof_s, te_s = run_seed_lr(sd)
    oofs.append(oof_s); tests.append(te_s)

oof_mean = np.mean(np.stack(oofs, axis=0), axis=0)
te_mean = np.mean(np.stack(tests, axis=0), axis=0)
auc = macro_auc_ignoring_degenerate(Y_train, oof_mean)
print(f'LR baseline OOF Macro AUC: {auc:.5f}')

# Save for potential blend/debug
np.save('prod_lr_oof.npy', oof_mean)
np.save('prod_lr_test.npy', te_mean)
np.save('prod_lr_train_ids.npy', rec_train)
np.save('prod_lr_test_ids.npy', rec_test)
print('Saved prod_lr_* files.')

Starting LogisticRegression sanity baseline...
== LR Seed 42 ==
[LR] Class 00 fold 3: degenerate -> const 0.0271
[LR] Class 01 fold 2: degenerate -> const 0.1085
[LR] Class 03 fold 1: degenerate -> const 0.0155
[LR] Class 03 fold 2: degenerate -> const 0.0155
[LR] Class 03 fold 3: degenerate -> const 0.0155
[LR] Class 11 fold 2: degenerate -> const 0.0388
[LR] Class 13 fold 1: degenerate -> const 0.0155
[LR] Class 13 fold 2: degenerate -> const 0.0155
[LR] Class 13 fold 3: degenerate -> const 0.0155
[LR] Class 15 fold 3: degenerate -> const 0.0233
[LR] Class 16 fold 1: degenerate -> const 0.0078
[LR] Class 16 fold 2: degenerate -> const 0.0078
[LR] Class 16 fold 3: degenerate -> const 0.0078
[LR] Class 17 fold 2: degenerate -> const 0.0155
[LR] Class 18 fold 2: degenerate -> const 0.0465


LR baseline OOF Macro AUC: 0.64642
Saved prod_lr_* files.


In [27]:
# Diagnostics: alignment assertions, prevalence, and simple station-prior baseline
import numpy as np, pandas as pd, random
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

print('=== DIAGNOSTICS ===')
# 1) Class index check from label_map
all_labels = []
for rid, y in label_map.items():
    idxs = np.where(np.asarray(y, dtype=np.int8)==1)[0].tolist()
    all_labels.extend(idxs)
if len(all_labels):
    mn, mx = int(np.min(all_labels)), int(np.max(all_labels))
    print(f'Label index range in label_map: [{mn}, {mx}] (num_classes={num_classes})')
    assert 0 <= mn and mx < num_classes, 'Class indices out of range!'
else:
    print('No positive labels found in label_map (unexpected)')

# 2) Prevalence per class in Y_train
pos_counts = Y_train.sum(axis=0).astype(int)
print('Positives per class in Y_train:', pos_counts.tolist())

# 3) GroupKFold prevalence (ensure positives in some folds)
gkf = GroupKFold(n_splits=3)
for c in range(num_classes):
    y = Y_train[:, c].astype(int)
    prev = []
    for tr, va in gkf.split(X_base, y, groups):
        prev.append(int(y[va].sum()))
    print(f'Class {c:02d} valid positives per fold: {prev}')

# 4) Station-prior-only OOF baseline
def station_prior_oof(y_full):
    oof = np.zeros_like(y_full, dtype=np.float32)
    for c in range(y_full.shape[1]):
        y = y_full[:, c].astype(int)
        cls_oof = np.zeros(y.shape[0], dtype=np.float32)
        for tr, va in gkf.split(X_base, y, groups):
            st_tr = groups[tr]; st_va = groups[va]
            y_tr = y[tr]
            alpha = 5.0; gm = float(y_tr.mean()) if y_tr.size>0 else 0.0
            st_prior = {}
            for s in np.unique(st_tr):
                m = (st_tr == s); cnt = int(m.sum()); sm = float(y_tr[m].sum()) if cnt>0 else 0.0
                st_prior[s] = (sm + alpha*gm) / (cnt + alpha)
            cls_oof[va] = np.array([st_prior.get(s, gm) for s in st_va], dtype=np.float32)
        oof[:, c] = cls_oof
    return oof

oof_prior = station_prior_oof(Y_train)
auc_prior = macro_auc_ignoring_degenerate(Y_train, oof_prior)
print(f'Station-prior-only OOF Macro AUC: {auc_prior:.5f}')

# 5) Shuffle-control: shuffle labels rows and compute macro AUC on station prior preds (should ~0.5)
rng = np.random.default_rng(123)
Y_shuf = Y_train.copy()
rng.shuffle(Y_shuf, axis=0)
auc_shuf = macro_auc_ignoring_degenerate(Y_shuf, oof_prior)
print(f'Shuffle-control Macro AUC (priors vs shuffled labels): {auc_shuf:.5f}')

# 6) Saved OOF/test shape checks for latest XGB/LR/LGB (if exist)
def check_saved(prefix):
    try:
        o = np.load(f'{prefix}_oof.npy'); t = np.load(f'{prefix}_test.npy')
        tri = np.load(f'{prefix}_train_ids.npy'); tei = np.load(f'{prefix}_test_ids.npy')
        print(f'{prefix}: OOF {o.shape}, TEST {t.shape}, train_ids {tri.shape}, test_ids {tei.shape}')
        assert o.shape == Y_train.shape, f'{prefix} OOF shape mismatch with Y_train!'
    except Exception as e:
        print(f'{prefix}: not checked ({e})')

for pref in ['prod_xgb','prod_lr','prod_lgb']:
    check_saved(pref)

print('=== END DIAGNOSTICS ===')

=== DIAGNOSTICS ===
Label index range in label_map: [0, 18] (num_classes=19)
Positives per class in Y_train: [7, 28, 19, 4, 9, 5, 19, 19, 24, 21, 49, 10, 12, 4, 16, 6, 2, 4, 12]
Class 00 valid positives per fold: [4, 3, 0]
Class 01 valid positives per fold: [24, 0, 4]
Class 02 valid positives per fold: [7, 5, 7]
Class 03 valid positives per fold: [0, 0, 4]
Class 04 valid positives per fold: [1, 7, 1]
Class 05 valid positives per fold: [2, 1, 2]
Class 06 valid positives per fold: [1, 9, 9]
Class 07 valid positives per fold: [8, 5, 6]
Class 08 valid positives per fold: [3, 10, 11]
Class 09 valid positives per fold: [3, 10, 8]
Class 10 valid positives per fold: [23, 11, 15]
Class 11 valid positives per fold: [9, 0, 1]
Class 12 valid positives per fold: [5, 3, 4]
Class 13 valid positives per fold: [0, 4, 0]
Class 14 valid positives per fold: [2, 11, 3]
Class 15 valid positives per fold: [1, 5, 0]
Class 16 valid positives per fold: [0, 2, 0]
Class 17 valid positives per fold: [3, 0, 1]
Clas

In [28]:
# Extra diagnostics: station prevalence sanity checks
import numpy as np
from sklearn.metrics import roc_auc_score

print('=== EXTRA DIAGNOSTICS: Station prevalence sanity ===')
stations_tr = groups  # aligned with Y_train/rec_train
assert len(stations_tr) == Y_train.shape[0], 'stations_tr misaligned with Y_train'

def auc_ignore_deg(y, p):
    if y.sum()==0 or y.sum()==y.shape[0]:
        return np.nan
    return roc_auc_score(y, p)

# 1) Global (leaky) per-station prevalence AUC on same data. Should be high if alignment OK.
aucs = []
for c in range(Y_train.shape[1]):
    y = Y_train[:, c].astype(int)
    # compute prevalence per station using ALL training rows (leaky by design)
    prev_map = {}
    for s in np.unique(stations_tr):
        m = (stations_tr == s)
        prev_map[s] = float(y[m].mean()) if m.any() else 0.0
    p = np.array([prev_map[s] for s in stations_tr], dtype=np.float32)
    auc = auc_ignore_deg(y, p)
    aucs.append(auc)
macro_auc_global = float(np.nanmean([a for a in aucs if not np.isnan(a)])) if aucs else float('nan')
print('Global (leaky) station-prevalence Macro AUC:', f'{macro_auc_global:.5f}')

# 2) Per-fold non-leaky station prevalence AUC (duplicate of prior but print correlations too)
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(n_splits=3)
aucs_cv = []
for c in range(Y_train.shape[1]):
    y = Y_train[:, c].astype(int)
    cls_oof = np.zeros_like(y, dtype=np.float32)
    for tr, va in gkf.split(np.zeros((Y_train.shape[0], 1)), y, stations_tr):
        st_tr = stations_tr[tr]; st_va = stations_tr[va]
        y_tr = y[tr]
        prev_map = {}
        for s in np.unique(st_tr):
            m = (st_tr == s)
            prev_map[s] = float(y_tr[m].mean()) if m.any() else 0.0
        cls_oof[va] = np.array([prev_map.get(s, float(y_tr.mean())) for s in st_va], dtype=np.float32)
    auc = auc_ignore_deg(y, cls_oof)
    aucs_cv.append(auc)
macro_auc_cv = float(np.nanmean([a for a in aucs_cv if not np.isnan(a)])) if aucs_cv else float('nan')
print('CV station-prevalence Macro AUC:', f'{macro_auc_cv:.5f}')

# 3) Print top/bottom stations by prevalence for a common class (e.g., class 10)
c = 10
y = Y_train[:, c].astype(int)
prev_map = {}
for s in np.unique(stations_tr):
    m = (stations_tr == s)
    prev_map[s] = (float(y[m].mean()) if m.any() else 0.0, int(m.sum()))
tops = sorted(prev_map.items(), key=lambda kv: kv[1][0], reverse=True)[:5]
bots = sorted(prev_map.items(), key=lambda kv: kv[1][0])[:5]
print('Class 10 top stations (prev, count):', tops)
print('Class 10 bottom stations (prev, count):', bots)
print('=== END EXTRA DIAGNOSTICS ===')

=== EXTRA DIAGNOSTICS: Station prevalence sanity ===
Global (leaky) station-prevalence Macro AUC: 0.86489
CV station-prevalence Macro AUC: 0.30427
Class 10 top stations (prev, count): [('PC4', (0.6, 15)), ('PC2', (0.3333333333333333, 15)), ('PC5', (0.3, 20)), ('PC18', (0.2727272727272727, 11)), ('PC1', (0.25925925925925924, 27))]
Class 10 bottom stations (prev, count): [('PC13', (0.0, 14)), ('PC10', (0.041666666666666664, 24)), ('PC11', (0.07692307692307693, 26)), ('PC8', (0.1, 10)), ('PC15', (0.12, 25))]
=== END EXTRA DIAGNOSTICS ===


In [29]:
# Build GLOBAL station priors (using all training rows) and evaluate
import numpy as np
print('=== GLOBAL STATION PRIORS ===')
stations_tr = groups.astype(str)
stations_te = stations_test.astype(str)
n_tr, C = Y_train.shape
uniq_st = np.unique(stations_tr)
global_priors = {c: {} for c in range(C)}
for c in range(C):
    y = Y_train[:, c].astype(int)
    for s in uniq_st:
        m = (stations_tr == s)
        global_priors[c][s] = float(y[m].mean()) if m.any() else float(y.mean())

# Build matrices
ST_PRIOR_TRAIN = np.zeros((n_tr, C), dtype=np.float32)
for c in range(C):
    ST_PRIOR_TRAIN[:, c] = np.array([global_priors[c].get(s, float(Y_train[:, c].mean())) for s in stations_tr], dtype=np.float32)
ST_PRIOR_TEST = np.zeros((stations_te.shape[0], C), dtype=np.float32)
for c in range(C):
    gm = float(Y_train[:, c].mean())
    ST_PRIOR_TEST[:, c] = np.array([global_priors[c].get(s, gm) for s in stations_te], dtype=np.float32)

auc_global = macro_auc_ignoring_degenerate(Y_train, ST_PRIOR_TRAIN)
print(f'Global station-prior (leaky) Macro AUC: {auc_global:.5f}')
np.save('prod_stprior_train.npy', ST_PRIOR_TRAIN)
np.save('prod_stprior_test.npy', ST_PRIOR_TEST)
print('Saved prod_stprior_* npy files.')
print('=== END GLOBAL STATION PRIORS ===')

=== GLOBAL STATION PRIORS ===
Global station-prior (leaky) Macro AUC: 0.86489
Saved prod_stprior_* npy files.
=== END GLOBAL STATION PRIORS ===


In [30]:
# Minimal baseline: Logistic Regression on histogram-only (no station prior), GroupKFold by station
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

print('Starting LR histogram-only baseline...')
gkf = GroupKFold(n_splits=3)
n_tr, C = Y_train.shape
n_te = H_test.shape[0]

def run_lr_hist(seed=42, use_svd=True, n_comp=24):
    oof = np.zeros((n_tr, C), dtype=np.float32)
    te = np.zeros((n_te, C), dtype=np.float32)
    for c in range(C):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        fold_no = 0
        for tr, va in gkf.split(H_train, y, groups):
            fold_no += 1
            X_tr = H_train[tr]; X_va = H_train[va]; X_te = H_test
            y_tr = y[tr]; y_va = y[va]
            # Degenerate handling
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va] = const
                cls_te_acc += np.full(n_te, const, np.float32) / gkf.get_n_splits()
                continue
            # Optional SVD for stability
            Z_tr = X_tr; Z_va = X_va; Z_te = X_te
            if use_svd:
                k = int(min(n_comp, max(2, min(X_tr.shape[1], max(2, X_tr.shape[0]-1)))))
                if k >= 2:
                    svd = TruncatedSVD(n_components=k, random_state=seed)
                    svd.fit(X_tr)
                    Z_tr = svd.transform(X_tr).astype(np.float32)
                    Z_va = svd.transform(X_va).astype(np.float32)
                    Z_te = svd.transform(X_te).astype(np.float32)
            # Scale
            scaler = StandardScaler(with_mean=True, with_std=True)
            Z_tr_s = scaler.fit_transform(Z_tr)
            Z_va_s = scaler.transform(Z_va)
            Z_te_s = scaler.transform(Z_te)
            # LR
            lr = LogisticRegression(penalty='l2', C=0.5, class_weight='balanced', solver='liblinear', max_iter=1000, random_state=seed)
            lr.fit(Z_tr_s, y_tr)
            cls_oof[va] = lr.predict_proba(Z_va_s)[:,1].astype(np.float32)
            cls_te_acc += lr.predict_proba(Z_te_s)[:,1].astype(np.float32) / gkf.get_n_splits()
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

oof_lr_hist, te_lr_hist = run_lr_hist(seed=42, use_svd=True, n_comp=24)
auc_lr_hist = macro_auc_ignoring_degenerate(Y_train, oof_lr_hist)
print(f'LR histogram-only OOF Macro AUC: {auc_lr_hist:.5f}')

np.save('prod_hist_lr_oof.npy', oof_lr_hist)
np.save('prod_hist_lr_test.npy', te_lr_hist)
np.save('prod_hist_lr_train_ids.npy', rec_train)
np.save('prod_hist_lr_test_ids.npy', rec_test)
print('Saved prod_hist_lr_* files.')

Starting LR histogram-only baseline...


LR histogram-only OOF Macro AUC: 0.62117
Saved prod_hist_lr_* files.


In [35]:
# XGBoost with histogram + GLOBAL station prior (leaky for OOF, strong for LB); GroupKFold by station
import sys, subprocess, time, os
from pathlib import Path
try:
    import xgboost as xgb
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import TruncatedSVD

print('Starting XGBoost + GLOBAL station prior... xgb version:', getattr(xgb, '__version__', 'unknown'))

# Load global station priors computed in cell 9
ST_PRIOR_TRAIN = np.load('prod_stprior_train.npy')  # shape (n_tr, C)
ST_PRIOR_TEST = np.load('prod_stprior_test.npy')    # shape (n_te, C)

gkf = GroupKFold(n_splits=3)
n_tr = Y_train.shape[0]
n_te = H_test.shape[0]

def run_seed_xgb_global(seed):
    oof = np.zeros((n_tr, num_classes), dtype=np.float32)
    te = np.zeros((n_te, num_classes), dtype=np.float32)
    for c in range(num_classes):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        fold_no = 0
        for tr_idx, va_idx in gkf.split(H_train, y, groups):
            fold_no += 1
            tstart = time.time()
            Xh_tr = H_train[tr_idx]; Xh_va = H_train[va_idx]; Xh_te = H_test
            y_tr = y[tr_idx]; y_va = y[va_idx]
            # Attach GLOBAL station prior for this class (1 feature)
            st_tr_feat = ST_PRIOR_TRAIN[tr_idx, c][:, None].astype(np.float32)
            st_va_feat = ST_PRIOR_TRAIN[va_idx, c][:, None].astype(np.float32)
            st_te_feat = ST_PRIOR_TEST[:, c][:, None].astype(np.float32)
            # Per-fold SVD for stability
            n_comp = int(min(24, max(2, min(Xh_tr.shape[1], max(2, Xh_tr.shape[0]-1))))) if Xh_tr.size>0 else 0
            if n_comp >= 2:
                svd = TruncatedSVD(n_components=n_comp, random_state=seed)
                svd.fit(Xh_tr)
                Z_tr = svd.transform(Xh_tr).astype(np.float32)
                Z_va = svd.transform(Xh_va).astype(np.float32)
                Z_te = svd.transform(Xh_te).astype(np.float32)
            else:
                Z_tr = None; Z_va = None; Z_te = None
            # Build matrices: [GLOBAL prior] + SVD + RAW hist
            parts_tr = [st_tr_feat]
            parts_va = [st_va_feat]
            parts_te = [st_te_feat]
            if Z_tr is not None:
                parts_tr.append(Z_tr); parts_va.append(Z_va); parts_te.append(Z_te)
            parts_tr.append(Xh_tr); parts_va.append(Xh_va); parts_te.append(Xh_te)
            X_tr_mat = np.concatenate(parts_tr, axis=1).astype(np.float32)
            X_va_mat = np.concatenate(parts_va, axis=1).astype(np.float32)
            X_te_mat = np.concatenate(parts_te, axis=1).astype(np.float32)
            # Degenerate handling
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_te, const, np.float32) / gkf.get_n_splits()
                print(f'[XGB-G] Class {c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-tstart:.2f}s', flush=True)
                continue
            spw = neg / max(1, pos)
            dtrain = xgb.DMatrix(X_tr_mat, label=y_tr)
            dvalid = xgb.DMatrix(X_va_mat, label=y_va)
            dtest = xgb.DMatrix(X_te_mat)
            params = {
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'max_depth': 4,
                'min_child_weight': 1e-3,
                'eta': 0.03,
                'subsample': 0.9,
                'colsample_bytree': 0.8,
                'lambda': 0.1,
                'tree_method': 'hist',
                'max_bin': 64,
                'scale_pos_weight': float(spw),
                'seed': int(seed),
                'verbosity': 0
            }
            bst = xgb.train(params, dtrain, num_boost_round=4000, evals=[(dvalid, 'valid')],
                            early_stopping_rounds=200, verbose_eval=200)
            p_va = bst.predict(dvalid, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            p_te = bst.predict(dtest, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / gkf.get_n_splits()
            print(f'[XGB-G] Class {c:02d} fold {fold_no}: pos={pos} neg={neg} spw={spw:.2f} best_iter={bst.best_iteration} | {time.time()-tstart:.2f}s', flush=True)
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

seeds = [42, 7, 2025]
oofs = []; tests = []
for i, sd in enumerate(seeds, 1):
    print(f'== XGB-G Seed {sd} ({i}/{len(seeds)}) ==', flush=True)
    oof_s, te_s = run_seed_xgb_global(sd)
    oofs.append(oof_s); tests.append(te_s)

oof_mean = np.mean(np.stack(oofs, axis=0), axis=0)
te_mean = np.mean(np.stack(tests, axis=0), axis=0)
auc = macro_auc_ignoring_degenerate(Y_train, oof_mean)
print(f'Final XGB + GLOBAL prior (3-seed) OOF Macro AUC: {auc:.5f}')

np.save('prod_xgbG_oof.npy', oof_mean)
np.save('prod_xgbG_test.npy', te_mean)
np.save('prod_xgbG_train_ids.npy', rec_train)
np.save('prod_xgbG_test_ids.npy', rec_test)
print('Saved prod_xgbG_* files.')

Starting XGBoost + GLOBAL station prior... xgb version: 2.1.4
== XGB-G Seed 42 (1/3) ==


[0]	valid-auc:0.66092


[200]	valid-auc:0.94828


[238]	valid-auc:0.94828


[XGB-G] Class 00 fold 1: pos=3 neg=164 spw=54.67 best_iter=39 | 0.32s


...

[0]	valid-auc:0.63173


[200]	valid-auc:0.93878


[226]	valid-auc:0.93321


[XGB-G] Class 18 fold 3: pos=5 neg=169 spw=33.80 best_iter=26 | 0.27s


Final XGB + GLOBAL prior (3-seed) OOF Macro AUC: 0.63110
Saved prod_xgbG_* files.


In [36]:
# Evaluate saved prod_xgbG OOF and build submission
import numpy as np, pandas as pd

oof = np.load('prod_xgbG_oof.npy')
te = np.load('prod_xgbG_test.npy')
tr_ids = np.load('prod_xgbG_train_ids.npy').astype(int)
te_ids = np.load('prod_xgbG_test_ids.npy').astype(int)

# Build Y in the same order as tr_ids using label_map from setup cell
Y_eval = np.vstack([label_map[int(r)] for r in tr_ids]).astype(np.int8)
auc = macro_auc_ignoring_degenerate(Y_eval, oof)
print(f'prod_xgbG OOF Macro AUC (ID-aligned): {auc:.5f}')

# Create submission aligned to sample_submission Id encoding (rec_id*100+class_id)
sub = pd.read_csv('sample_submission.csv')
id_vals = sub['Id'].values.astype(int)
rid_to_idx = {rid:i for i, rid in enumerate(te_ids)}
probs = np.zeros_like(id_vals, dtype=np.float32)
for i, Id in enumerate(id_vals):
    rid = Id // 100; cid = Id % 100
    row = rid_to_idx.get(rid, None)
    p = float(te[row, cid]) if (row is not None and cid < num_classes) else 0.0
    probs[i] = np.clip(p, 1e-6, 1-1e-6)
sub['Probability'] = probs
sub.to_csv('submission_prod_xgbG.csv', index=False)
print('Saved submission_prod_xgbG.csv')

prod_xgbG OOF Macro AUC (ID-aligned): 0.63110
Saved submission_prod_xgbG.csv


In [43]:
# Copy best submission to submission.csv for scoring
import pandas as pd
sub = pd.read_csv('submission_xgbB.csv')
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv from submission_xgbB.csv')

Wrote submission.csv from submission_xgbB.csv


In [40]:
# LR on Hellinger-transformed hist + curated features (non-leaky), GroupKFold by station
import numpy as np, pandas as pd, time
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

print('Starting LR on Hellinger-hist + curated features...')
gkf = GroupKFold(n_splits=3)
n_tr, C = Y_train.shape
n_te = H_test_h.shape[0]

def run_lr_hc(seed=42, use_svd=True, n_comp=32, C_reg=0.8):
    oof = np.zeros((n_tr, C), dtype=np.float32)
    te = np.zeros((n_te, C), dtype=np.float32)
    for c in range(C):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        fold_no = 0
        for tr, va in gkf.split(H_train_h, y, groups):
            fold_no += 1
            t0 = time.time()
            Xh_tr = H_train_h[tr]; Xh_va = H_train_h[va]; Xh_te = H_test_h
            Xc_tr = C_train[tr];   Xc_va = C_train[va];   Xc_te = C_test
            y_tr = y[tr]; y_va = y[va]
            # Degenerate handling
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va] = const
                cls_te_acc += np.full(n_te, const, np.float32) / gkf.get_n_splits()
                continue
            # Per-fold SVD on Hellinger-hist
            parts_tr = []; parts_va = []; parts_te = [];
            if use_svd:
                k = int(min(n_comp, max(2, min(Xh_tr.shape[1], max(2, Xh_tr.shape[0]-1)))))
                if k >= 2:
                    svd = TruncatedSVD(n_components=k, random_state=seed)
                    svd.fit(Xh_tr)
                    Z_tr = svd.transform(Xh_tr).astype(np.float32)
                    Z_va = svd.transform(Xh_va).astype(np.float32)
                    Z_te = svd.transform(Xh_te).astype(np.float32)
                    parts_tr.append(Z_tr); parts_va.append(Z_va); parts_te.append(Z_te)
                else:
                    parts_tr.append(Xh_tr); parts_va.append(Xh_va); parts_te.append(Xh_te)
            else:
                parts_tr.append(Xh_tr); parts_va.append(Xh_va); parts_te.append(Xh_te)
            # Append curated features
            parts_tr.append(Xc_tr); parts_va.append(Xc_va); parts_te.append(Xc_te)
            X_tr_mat = np.concatenate(parts_tr, axis=1).astype(np.float32)
            X_va_mat = np.concatenate(parts_va, axis=1).astype(np.float32)
            X_te_mat = np.concatenate(parts_te, axis=1).astype(np.float32)
            # Scale per-fold
            scaler = StandardScaler(with_mean=True, with_std=True)
            X_tr_s = scaler.fit_transform(X_tr_mat)
            X_va_s = scaler.transform(X_va_mat)
            X_te_s = scaler.transform(X_te_mat)
            # Train LR
            lr = LogisticRegression(penalty='l2', C=C_reg, class_weight='balanced', solver='liblinear', max_iter=2000, random_state=seed)
            lr.fit(X_tr_s, y_tr)
            cls_oof[va] = lr.predict_proba(X_va_s)[:,1].astype(np.float32)
            cls_te_acc += lr.predict_proba(X_te_s)[:,1].astype(np.float32) / gkf.get_n_splits()
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

oof_lr_hc, te_lr_hc = run_lr_hc(seed=42, use_svd=True, n_comp=32, C_reg=0.8)
auc_lr_hc = macro_auc_ignoring_degenerate(Y_train, oof_lr_hc)
print(f'LR Hellinger+Curated OOF Macro AUC: {auc_lr_hc:.5f}')

np.save('prod_lr_hc_oof.npy', oof_lr_hc)
np.save('prod_lr_hc_test.npy', te_lr_hc)
np.save('prod_lr_hc_train_ids.npy', rec_train)
np.save('prod_lr_hc_test_ids.npy', rec_test)
print('Saved prod_lr_hc_* files.')

# Build submission for quick LB probe
sub = pd.read_csv('sample_submission.csv')
id_vals = sub['Id'].values.astype(int)
rid_to_idx = {rid:i for i, rid in enumerate(rec_test)}
probs = np.zeros_like(id_vals, dtype=np.float32)
for i, Id in enumerate(id_vals):
    rid = Id // 100; cid = Id % 100
    row = rid_to_idx.get(rid, None)
    p = float(te_lr_hc[row, cid]) if (row is not None and cid < num_classes) else 0.0
    probs[i] = np.clip(p, 1e-6, 1-1e-6)
sub['Probability'] = probs
sub.to_csv('submission_lr_hc.csv', index=False)
print('Saved submission_lr_hc.csv')

Starting LR on Hellinger-hist + curated features...


LR Hellinger+Curated OOF Macro AUC: 0.58278
Saved prod_lr_hc_* files.
Saved submission_lr_hc.csv


In [42]:
# Path B: XGBoost with Hellinger hist + curated feats + GLOBAL station prior; 5-fold KFold (leaky leaderboard probe)
import sys, subprocess, time, os
try:
    import xgboost as xgb
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
import numpy as np, pandas as pd

print('Starting XGB Path-B (KFold, Hellinger+Curated+GLOBAL prior)...')

# Require global station priors and prepared features from cell 1 & 9
ST_PRIOR_TRAIN = np.load('prod_stprior_train.npy')
ST_PRIOR_TEST = np.load('prod_stprior_test.npy')

n_tr = Y_train.shape[0]
n_te = H_test_h.shape[0]

def run_seed(seed=42, n_splits=5, use_svd=True, n_comp=32):
    oof = np.zeros((n_tr, num_classes), dtype=np.float32)
    te = np.zeros((n_te, num_classes), dtype=np.float32)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    # Use same splits for all classes for speed
    idx_folds = list(kf.split(H_train_h))
    for c in range(num_classes):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        for fold_no, (tr_idx, va_idx) in enumerate(idx_folds, 1):
            t0 = time.time()
            Xh_tr = H_train_h[tr_idx]; Xh_va = H_train_h[va_idx]; Xh_te = H_test_h
            Xc_tr = C_train[tr_idx];   Xc_va = C_train[va_idx];   Xc_te = C_test
            y_tr = y[tr_idx]; y_va = y[va_idx]
            # Attach GLOBAL station prior for this class (1 feature)
            st_tr_feat = ST_PRIOR_TRAIN[tr_idx, c][:, None].astype(np.float32)
            st_va_feat = ST_PRIOR_TRAIN[va_idx, c][:, None].astype(np.float32)
            st_te_feat = ST_PRIOR_TEST[:, c][:, None].astype(np.float32)
            # Per-fold SVD on Hellinger hist
            parts_tr = [st_tr_feat]; parts_va = [st_va_feat]; parts_te = [st_te_feat]
            if use_svd and Xh_tr.size>0:
                k = int(min(n_comp, max(2, min(Xh_tr.shape[1], max(2, Xh_tr.shape[0]-1)))))
                if k >= 2:
                    svd = TruncatedSVD(n_components=k, random_state=seed)
                    svd.fit(Xh_tr)
                    Z_tr = svd.transform(Xh_tr).astype(np.float32)
                    Z_va = svd.transform(Xh_va).astype(np.float32)
                    Z_te = svd.transform(Xh_te).astype(np.float32)
                    parts_tr.append(Z_tr); parts_va.append(Z_va); parts_te.append(Z_te)
                else:
                    parts_tr.append(Xh_tr); parts_va.append(Xh_va); parts_te.append(Xh_te)
            else:
                parts_tr.append(Xh_tr); parts_va.append(Xh_va); parts_te.append(Xh_te)
            # Append curated features
            parts_tr.append(Xc_tr); parts_va.append(Xc_va); parts_te.append(Xc_te)
            X_tr_mat = np.concatenate(parts_tr, axis=1).astype(np.float32)
            X_va_mat = np.concatenate(parts_va, axis=1).astype(np.float32)
            X_te_mat = np.concatenate(parts_te, axis=1).astype(np.float32)
            # Degenerate handling
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_te, const, np.float32) / n_splits
                print(f'[XGB-B] c={c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-t0:.2f}s', flush=True)
                continue
            spw = neg / max(1, pos)
            dtrain = xgb.DMatrix(X_tr_mat, label=y_tr)
            dvalid = xgb.DMatrix(X_va_mat, label=y_va)
            dtest = xgb.DMatrix(X_te_mat)
            params = {
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'max_depth': 4,
                'min_child_weight': 1e-3,
                'eta': 0.03,
                'subsample': 0.9,
                'colsample_bytree': 0.8,
                'lambda': 0.1,
                'tree_method': 'hist',
                'max_bin': 64,
                'scale_pos_weight': float(spw),
                'seed': int(seed),
                'verbosity': 0
            }
            bst = xgb.train(params, dtrain, num_boost_round=5000, evals=[(dvalid, 'valid')],
                            early_stopping_rounds=200, verbose_eval=250)
            p_va = bst.predict(dvalid, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            p_te = bst.predict(dtest, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / n_splits
            print(f'[XGB-B] c={c:02d} fold {fold_no}: pos={pos} neg={neg} spw={spw:.2f} best_iter={bst.best_iteration} | {time.time()-t0:.2f}s', flush=True)
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

seeds = [42, 7, 2025]
oofs = []; tests = []
for i, sd in enumerate(seeds, 1):
    print(f'== XGB-B Seed {sd} ({i}/{len(seeds)}) ==', flush=True)
    oof_s, te_s = run_seed(seed=sd, n_splits=5, use_svd=True, n_comp=32)
    oofs.append(oof_s); tests.append(te_s)

oof_mean = np.mean(np.stack(oofs, axis=0), axis=0)
te_mean = np.mean(np.stack(tests, axis=0), axis=0)
auc = macro_auc_ignoring_degenerate(Y_train, oof_mean)
print(f'Final XGB Path-B (3-seed, 5-fold) OOF Macro AUC: {auc:.5f}')

np.save('prod_xgbB_oof.npy', oof_mean)
np.save('prod_xgbB_test.npy', te_mean)
np.save('prod_xgbB_train_ids.npy', rec_train)
np.save('prod_xgbB_test_ids.npy', rec_test)
print('Saved prod_xgbB_* files.')

# Build submission
sub = pd.read_csv('sample_submission.csv')
id_vals = sub['Id'].values.astype(int)
rid_to_idx = {rid:i for i, rid in enumerate(rec_test)}
probs = np.zeros_like(id_vals, dtype=np.float32)
for i, Id in enumerate(id_vals):
    rid = Id // 100; cid = Id % 100
    row = rid_to_idx.get(rid, None)
    p = float(te_mean[row, cid]) if (row is not None and cid < num_classes) else 0.0
    probs[i] = np.clip(p, 1e-6, 1-1e-6)
sub['Probability'] = probs
sub.to_csv('submission_xgbB.csv', index=False)
print('Saved submission_xgbB.csv')

Starting XGB Path-B (KFold, Hellinger+Curated+GLOBAL prior)...
== XGB-B Seed 42 (1/3) ==


[0]	valid-auc:0.99500


[201]	valid-auc:1.00000


[XGB-B] c=00 fold 1: pos=5 neg=201 spw=40.20 best_iter=1 | 0.33s


[0]	valid-auc:0.37000


...

[XGB-B] c=18 fold 4: pos=11 neg=196 spw=17.82 best_iter=87 | 0.51s


[0]	valid-auc:0.82292


[204]	valid-auc:0.95833


[XGB-B] c=18 fold 5: pos=9 neg=198 spw=22.00 best_iter=5 | 0.49s


Final XGB Path-B (3-seed, 5-fold) OOF Macro AUC: 0.79066
Saved prod_xgbB_* files.
Saved submission_xgbB.csv


In [44]:
# Blend multiple model predictions and write submissions (mean and rank-mean)
import numpy as np, pandas as pd
from pathlib import Path

print('=== BLENDING SUBMISSIONS ===')
rec_test_ref = np.load('prod_test_ids.npy').astype(int)
num_classes_local = int(num_classes)

def load_preds(prefix):
    te_fp = Path(f'{prefix}_test.npy')
    ids_fp = Path(f'{prefix}_test_ids.npy')
    if not te_fp.exists() or not ids_fp.exists():
        print(f'.. skip {prefix}: files not found')
        return None
    te = np.load(te_fp)
    ids = np.load(ids_fp).astype(int)
    if te.shape[1] != num_classes_local:
        print(f'.. skip {prefix}: class mismatch {te.shape}')
        return None
    # align to rec_test_ref order
    idx_map = {rid:i for i, rid in enumerate(ids)}
    try:
        aligned = np.vstack([te[idx_map[r]] for r in rec_test_ref])
    except KeyError as e:
        print(f'.. skip {prefix}: missing rid {e}')
        return None
    print(f'.. loaded {prefix}: {aligned.shape}')
    return aligned.astype(np.float32)

candidates = [
    'prod_xgbB',   # KFold Path-B (leaky) strong LB
    'prod_xgbG',   # GroupKFold + GLOBAL prior
    'prod_xgb',    # GroupKFold minimal hist
    'prod_hist_lr',# LR hist-only
    'prod_lr_hc'   # LR Hellinger+Curated
]
pred_list = []
for pref in candidates:
    arr = load_preds(pref)
    if arr is not None:
        pred_list.append(arr)

assert len(pred_list) >= 2, 'Need at least two models to blend'
P = np.stack(pred_list, axis=0)  # (M, N, C)
P_mean = P.mean(axis=0)

# Rank-average per class
def rank_average(Pstack):
    M, N, C = Pstack.shape
    out = np.zeros((N, C), dtype=np.float32)
    for c in range(C):
        ranks = np.zeros((M, N), dtype=np.float32)
        for m in range(M):
            p = Pstack[m, :, c]
            order = np.argsort(np.argsort(p))  # 0..N-1
            ranks[m] = order.astype(np.float32) / max(1, N-1)
        out[:, c] = ranks.mean(axis=0)
    return out

P_rank = rank_average(P)

def write_submission(probs, fname):
    sub = pd.read_csv('sample_submission.csv')
    id_vals = sub['Id'].values.astype(int)
    rid_to_row = {rid:i for i, rid in enumerate(rec_test_ref)}
    out = np.zeros_like(id_vals, dtype=np.float32)
    for i, Id in enumerate(id_vals):
        rid = Id // 100; cid = Id % 100
        row = rid_to_row.get(rid, None)
        p = float(probs[row, cid]) if (row is not None and cid < num_classes_local) else 0.0
        out[i] = np.clip(p, 1e-6, 1-1e-6)
    sub['Probability'] = out
    sub.to_csv(fname, index=False)
    print('Wrote', fname)

write_submission(P_mean, 'submission_blend_mean.csv')
write_submission(P_rank, 'submission_blend_rank.csv')

# Default to rank blend for submission.csv
pd.read_csv('submission_blend_rank.csv').to_csv('submission.csv', index=False)
print('Also wrote submission.csv (rank blend)')
print('=== END BLENDING ===')

=== BLENDING SUBMISSIONS ===
.. loaded prod_xgbB: (64, 19)
.. loaded prod_xgbG: (64, 19)
.. loaded prod_xgb: (64, 19)
.. loaded prod_hist_lr: (64, 19)
.. loaded prod_lr_hc: (64, 19)
Wrote submission_blend_mean.csv
Wrote submission_blend_rank.csv
Also wrote submission.csv (rank blend)
=== END BLENDING ===


In [48]:
# FINAL PUSH v2: MultilabelStratifiedKFold, shallower trees, SVD(H_idf) features, rank-avg blend
import sys, subprocess, time, os, math
import numpy as np, pandas as pd

print('=== FINAL PUSH MODELS (MLSKFold, leaky for LB) ===')
# Ensure required packages
try:
    import xgboost as xgb
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'xgboost'])
    import xgboost as xgb
try:
    import lightgbm as lgb
    from lightgbm import LGBMClassifier
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'lightgbm'])
    import lightgbm as lgb
    from lightgbm import LGBMClassifier
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'iterative-stratification'])
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge

# Preconditions: materials from Cell 1 and 9
ST_PRIOR_TRAIN = np.load('prod_stprior_train.npy')
ST_PRIOR_TEST = np.load('prod_stprior_test.npy')

n_tr, C = Y_train.shape
n_te = C_test.shape[0]
assert H_tree_train.shape[0] == n_tr and H_tree_test.shape[0] == n_te
assert H_idf_train.shape[0] == n_tr and H_idf_test.shape[0] == n_te

# Helper: macro AUC already defined earlier as macro_auc_ignoring_degenerate

def build_folds(seed=42, n_splits=5):
    mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    return list(mskf.split(np.zeros((n_tr, 1)), Y_train))

def train_xgb_tree(seed=42, n_splits=5, k_svd=16):
    folds = build_folds(seed, n_splits)
    oof = np.zeros((n_tr, C), dtype=np.float32)
    te = np.zeros((n_te, C), dtype=np.float32)
    for c in range(C):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        for fold_no, (tr_idx, va_idx) in enumerate(folds, 1):
            t0 = time.time()
            # Features: H_tree + curated + GLOBAL prior (1d for this class) + SVD(H_idf, k)
            st_tr = ST_PRIOR_TRAIN[tr_idx, c][:, None]
            st_va = ST_PRIOR_TRAIN[va_idx, c][:, None]
            st_te = ST_PRIOR_TEST[:, c][:, None]
            # Per-fold SVD on H_idf
            if H_idf_train.shape[1] > 1 and len(tr_idx) > 2:
                k = int(min(k_svd, max(2, min(H_idf_train.shape[1], len(tr_idx)-1))))
            else:
                k = 0
            if k >= 2:
                svd = TruncatedSVD(n_components=k, random_state=seed)
                Z_tr = svd.fit_transform(H_idf_train[tr_idx]).astype(np.float32)
                Z_va = svd.transform(H_idf_train[va_idx]).astype(np.float32)
                Z_te = svd.transform(H_idf_test).astype(np.float32)
            else:
                Z_tr = np.empty((len(tr_idx), 0), dtype=np.float32)
                Z_va = np.empty((len(va_idx), 0), dtype=np.float32)
                Z_te = np.empty((n_te, 0), dtype=np.float32)
            X_tr = np.concatenate([H_tree_train[tr_idx], C_train[tr_idx], st_tr, Z_tr], axis=1).astype(np.float32)
            X_va = np.concatenate([H_tree_train[va_idx], C_train[va_idx], st_va, Z_va], axis=1).astype(np.float32)
            X_te = np.concatenate([H_tree_test, C_test, st_te, Z_te], axis=1).astype(np.float32)
            y_tr = y[tr_idx]; y_va = y[va_idx]
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_te, const, np.float32) / n_splits
                print(f'[XGB-TREE] c={c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-t0:.2f}s', flush=True)
                continue
            spw = neg / max(1, pos)
            dtr = xgb.DMatrix(X_tr, label=y_tr); dva = xgb.DMatrix(X_va, label=y_va); dte = xgb.DMatrix(X_te)
            params = {
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'max_depth': 3,
                'min_child_weight': 5.0,
                'eta': 0.04,
                'subsample': 0.75,
                'colsample_bytree': 0.8,
                'lambda': 1.5,
                'tree_method': 'hist',
                'max_bin': 64,
                'scale_pos_weight': float(spw),
                'seed': int(seed),
                'verbosity': 0
            }
            bst = xgb.train(params, dtr, num_boost_round=6000, evals=[(dva, 'valid')],
                            early_stopping_rounds=300, verbose_eval=250)
            p_va = bst.predict(dva, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            p_te = bst.predict(dte, iteration_range=(0, bst.best_iteration+1)).astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / n_splits
            print(f'[XGB-TREE] c={c:02d} fold {fold_no}: pos={pos} neg={neg} spw={spw:.2f} best_iter={bst.best_iteration} | {time.time()-t0:.2f}s', flush=True)
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

def train_lgbm_tree(seed=42, n_splits=5, k_svd=16):
    folds = build_folds(seed, n_splits)
    oof = np.zeros((n_tr, C), dtype=np.float32)
    te = np.zeros((n_te, C), dtype=np.float32)
    for c in range(C):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        for fold_no, (tr_idx, va_idx) in enumerate(folds, 1):
            t0 = time.time()
            st_tr = ST_PRIOR_TRAIN[tr_idx, c][:, None]
            st_va = ST_PRIOR_TRAIN[va_idx, c][:, None]
            st_te = ST_PRIOR_TEST[:, c][:, None]
            if H_idf_train.shape[1] > 1 and len(tr_idx) > 2:
                k = int(min(k_svd, max(2, min(H_idf_train.shape[1], len(tr_idx)-1))))
            else:
                k = 0
            if k >= 2:
                svd = TruncatedSVD(n_components=k, random_state=seed)
                Z_tr = svd.fit_transform(H_idf_train[tr_idx]).astype(np.float32)
                Z_va = svd.transform(H_idf_train[va_idx]).astype(np.float32)
                Z_te = svd.transform(H_idf_test).astype(np.float32)
            else:
                Z_tr = np.empty((len(tr_idx), 0), dtype=np.float32)
                Z_va = np.empty((len(va_idx), 0), dtype=np.float32)
                Z_te = np.empty((n_te, 0), dtype=np.float32)
            X_tr = np.concatenate([H_tree_train[tr_idx], C_train[tr_idx], st_tr, Z_tr], axis=1).astype(np.float32)
            X_va = np.concatenate([H_tree_train[va_idx], C_train[va_idx], st_va, Z_va], axis=1).astype(np.float32)
            X_te = np.concatenate([H_tree_test, C_test, st_te, Z_te], axis=1).astype(np.float32)
            y_tr = y[tr_idx]; y_va = y[va_idx]
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_te, const, np.float32) / n_splits
                print(f'[LGBM-TREE] c={c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-t0:.2f}s', flush=True)
                continue
            spw = neg / max(1, pos)
            model = LGBMClassifier(
                objective='binary',
                n_estimators=8000,
                learning_rate=0.04,
                num_leaves=23,
                max_depth=4,
                max_bin=64,
                subsample=0.75,
                subsample_freq=1,
                colsample_bytree=0.8,
                reg_lambda=1.5,
                min_child_samples=12,
                feature_pre_filter=False,
                force_col_wise=True,
                extra_trees=False,
                scale_pos_weight=spw,
                random_state=seed,
                n_jobs=-1
            )
            model.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], eval_metric='auc',
                      callbacks=[lgb.early_stopping(300, verbose=False), lgb.log_evaluation(100)])
            p_va = model.predict_proba(X_va)[:,1].astype(np.float32)
            p_te = model.predict_proba(X_te)[:,1].astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / n_splits
            print(f'[LGBM-TREE] c={c:02d} fold {fold_no}: pos={pos} neg={neg} spw={spw:.2f} best_iter={getattr(model, "best_iteration_", None)} | {time.time()-t0:.2f}s', flush=True)
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

def train_lr_idf(seed=42, n_splits=5, n_comp=64, C_reg=1.0):
    folds = build_folds(seed, n_splits)
    oof = np.zeros((n_tr, C), dtype=np.float32)
    te = np.zeros((n_te, C), dtype=np.float32)
    for c in range(C):
        y = Y_train[:, c].astype(int)
        cls_oof = np.zeros(n_tr, dtype=np.float32)
        cls_te_acc = np.zeros(n_te, dtype=np.float32)
        for fold_no, (tr_idx, va_idx) in enumerate(folds, 1):
            t0 = time.time()
            Xh_tr = H_idf_train[tr_idx]; Xh_va = H_idf_train[va_idx]; Xh_te = H_idf_test
            y_tr = y[tr_idx]; y_va = y[va_idx]
            pos = int(y_tr.sum()); neg = int((1-y_tr).sum())
            if pos == 0 or neg == 0 or y_va.sum() in (0, len(y_va)):
                const = float(y.mean())
                cls_oof[va_idx] = const
                cls_te_acc += np.full(n_te, const, np.float32) / n_splits
                print(f'[LR-IDF] c={c:02d} fold {fold_no}: degenerate -> const {const:.4f} | {time.time()-t0:.2f}s', flush=True)
                continue
            k = int(min(n_comp, max(2, min(Xh_tr.shape[1], max(2, Xh_tr.shape[0]-1)))))
            if k >= 2:
                svd = TruncatedSVD(n_components=k, random_state=seed)
                Z_tr = svd.fit_transform(Xh_tr).astype(np.float32)
                Z_va = svd.transform(Xh_va).astype(np.float32)
                Z_te = svd.transform(Xh_te).astype(np.float32)
            else:
                Z_tr, Z_va, Z_te = Xh_tr, Xh_va, Xh_te
            scaler = StandardScaler(with_mean=True, with_std=True)
            Z_tr_s = scaler.fit_transform(Z_tr)
            Z_va_s = scaler.transform(Z_va)
            Z_te_s = scaler.transform(Z_te)
            lr = LogisticRegression(penalty='l2', C=C_reg, class_weight='balanced', solver='liblinear', max_iter=2000, random_state=seed)
            lr.fit(Z_tr_s, y_tr)
            p_va = lr.predict_proba(Z_va_s)[:,1].astype(np.float32)
            p_te = lr.predict_proba(Z_te_s)[:,1].astype(np.float32)
            cls_oof[va_idx] = p_va
            cls_te_acc += p_te / n_splits
            print(f'[LR-IDF] c={c:02d} fold {fold_no}: pos={pos} neg={neg} k={k} | {time.time()-t0:.2f}s', flush=True)
        oof[:, c] = cls_oof
        te[:, c] = cls_te_acc
    return oof, te

def rank_average(Pstack):
    M, N, Cc = Pstack.shape
    out = np.zeros((N, Cc), dtype=np.float32)
    for c in range(Cc):
        ranks = np.zeros((M, N), dtype=np.float32)
        for m in range(M):
            p = Pstack[m, :, c]
            order = np.argsort(np.argsort(p))
            ranks[m] = order.astype(np.float32) / max(1, N-1)
        out[:, c] = ranks.mean(axis=0)
    return out

seeds = [42, 7]
xgb_oofs = []; xgb_tes = []
lgb_oofs = []; lgb_tes = []
lr_oofs = []; lr_tes = []

for i, sd in enumerate(seeds, 1):
    print(f'== SEED {sd} ({i}/{len(seeds)}) ==');
    t_seed = time.time()
    oof_x, te_x = train_xgb_tree(seed=sd, n_splits=5, k_svd=16)
    print(f'[SEED {sd}] XGB-TREE OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, oof_x):.5f}')
    xgb_oofs.append(oof_x); xgb_tes.append(te_x)

    oof_l, te_l = train_lgbm_tree(seed=sd, n_splits=5, k_svd=16)
    print(f'[SEED {sd}] LGBM-TREE OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, oof_l):.5f}')
    lgb_oofs.append(oof_l); lgb_tes.append(te_l)

    oof_r, te_r = train_lr_idf(seed=sd, n_splits=5, n_comp=64, C_reg=1.0)
    print(f'[SEED {sd}] LR-IDF OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, oof_r):.5f}')
    lr_oofs.append(oof_r); lr_tes.append(te_r)
    print(f'.. seed {sd} done in {time.time()-t_seed:.2f}s', flush=True)

# Aggregate across seeds
xgb_oof_mean = np.mean(np.stack(xgb_oofs, axis=0), axis=0)
xgb_te_mean = np.mean(np.stack(xgb_tes, axis=0), axis=0)
lgb_oof_mean = np.mean(np.stack(lgb_oofs, axis=0), axis=0)
lgb_te_mean = np.mean(np.stack(lgb_tes, axis=0), axis=0)
lr_oof_mean = np.mean(np.stack(lr_oofs, axis=0), axis=0)
lr_te_mean = np.mean(np.stack(lr_tes, axis=0), axis=0)

print(f'XGB-TREE (seed-avg) OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, xgb_oof_mean):.5f}')
print(f'LGBM-TREE (seed-avg) OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, lgb_oof_mean):.5f}')
print(f'LR-IDF (seed-avg) OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, lr_oof_mean):.5f}')

# Rank-average per class across models (seed-averaged predictions)
P_te_models = np.stack([xgb_te_mean, lgb_te_mean, lr_te_mean], axis=0)  # (3, N, C)
P_oof_models = np.stack([xgb_oof_mean, lgb_oof_mean, lr_oof_mean], axis=0)
blend_te_rank = rank_average(P_te_models)
blend_oof_rank = rank_average(P_oof_models)
print(f'Rank-Blend OOF Macro AUC: {macro_auc_ignoring_degenerate(Y_train, blend_oof_rank):.5f}')

# Save artifacts
np.save('prod_xgb_final_oof.npy', xgb_oof_mean); np.save('prod_xgb_final_test.npy', xgb_te_mean)
np.save('prod_lgb_final_oof.npy', lgb_oof_mean); np.save('prod_lgb_final_test.npy', lgb_te_mean)
np.save('prod_lr_idf_oof.npy', lr_oof_mean); np.save('prod_lr_idf_test.npy', lr_te_mean)
np.save('prod_final_train_ids.npy', rec_train); np.save('prod_final_test_ids.npy', rec_test)
np.save('prod_blend_final_oof.npy', blend_oof_rank); np.save('prod_blend_final_test.npy', blend_te_rank)
print('Saved final model npy files')

# Build submission.csv from rank blend
sub = pd.read_csv('sample_submission.csv')
id_vals = sub['Id'].values.astype(int)
rid_to_idx = {rid:i for i, rid in enumerate(rec_test)}
probs = np.zeros_like(id_vals, dtype=np.float32)
for i, Id in enumerate(id_vals):
    rid = Id // 100; cid = Id % 100
    row = rid_to_idx.get(rid, None)
    p = float(blend_te_rank[row, cid]) if (row is not None and cid < C) else 0.0
    probs[i] = np.clip(p, 1e-6, 1-1e-6)
sub['Probability'] = probs
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv (final rank blend)')
print('=== END FINAL PUSH v2 ===')